In [14]:
from flask import Flask, request, render_template, redirect,send_file, make_response
import requests
from pygame import mixer
from gtts import gTTS
import os
import win32com.client
# importing libraries 
import cv2 
import numpy as np 
import time
from PIL import Image, ImageTk
import numpy as np
import matplotlib.pyplot as plt
import _thread
import speech_recognition as sr 
import moviepy.editor as mp
import pytesseract
import matplotlib.pyplot as plt
import os 
import cv2
import numpy as np
import os
import sqlite3


AUDIO_FILE = r"C:\Users\nishc_omjn2ty\OneDrive\Desktop\nmit2021\Video_Transcription\sample.mp4"
# Create a VideoCapture object and read from input file 
cap = cv2.VideoCapture(AUDIO_FILE)
clip = mp.VideoFileClip(AUDIO_FILE) 
clip.audio.write_audiofile(r"converted.wav")
text=''

app = Flask(__name__,template_folder='templates')

@app.after_request
def add_header(response):
    # response.cache_control.no_store = True
    response.headers['Cache-Control'] = 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0, max-age=0'
    response.headers['Pragma'] = 'no-cache'
    response.headers['Expires'] = '-1'
    return response

@app.route('/')
def home():
    return render_template('login.html', msg='')

@app.route('/upload',methods=['GET','POST'])
def upload():
    if request.form['submit_button'] == 'LOGIN':
        if(request.form['username'] == '1234' and request.form['password'] == '1234' ):
            return redirect(('/home'))
        else:
            return render_template('login.html', msg="Invalid Credentials")
    if request.form['submit_button'] == 'FACE DETECTION LOGIN' :

        #path='/Users/sanjanasrinivasareddy/Desktop/nmit/20200219_111952.jpeg'
        facedetect = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

        #cam = cv2.VideoCapture(0)
        recognizer = cv2.face.LBPHFaceRecognizer_create()
        recognizer.read("trainingdata.yml")
        cam = cv2.VideoCapture(0)

        cv2.namedWindow("test")

        img_counter = 0
        img_name =''

        while True:
            ret, frame = cam.read()
            if not ret:
                print("failed to grab frame")
                break
            cv2.imshow("test", frame)
            k = cv2.waitKey(1)

            if k%256 == 32:
                # SPACE pressed
                img_name = "face.png"
                cv2.imwrite(img_name, frame)
                print("{} written!".format(img_name))
                img_counter += 1
                break

        cam.release()

        cv2.destroyAllWindows()
        # id = 0
        # font = cv2.cv.InitFont(cv2.cv.CV_FONT_HERSHEY_COMPLEX_SMALL,1,1,0,1)

        def getProfile(id):
            conn=sqlite3.connect("FaceBase.db")
            # cmd="SELECT * FROM Peoples WHERE ID="+str(id)
            cursor=conn.execute("SELECT * FROM Peoples WHERE id=?", (id,))
            profile=None
            for row in cursor:
                profile=row
            conn.close()
            return profile
        l=[]
        i=0

        #filelist = os.listdir(settings.path)
        try:
            xx=0

            file='face.png'
            print(file)

            img = cv2.imread(file)
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces=facedetect.detectMultiScale(gray,1.3,5);
            for(x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
                id,conf=recognizer.predict(gray[y:y+h,x:x+w])
                if(id==120):
                    id=4
                profile=getProfile(id)
                print(id)
                l.append(id)
            xx=xx+1
        except:
            xx=xx+1

        print(l)
        if(len(l)==0):
             return redirect(('/home'))
        else:
            p=max(set(l), key = l.count)
            profile=getProfile(p)
            settings.sad=profile
            print('student profile is')
            print(profile)
            return redirect(('/home'))


@app.route('/home',methods=['GET','POST'])
def end():
  # Load the popular external library
    return render_template('insta_tabs.html',message=text, l=0, display=0)


@app.route('/imagecaption',methods=['GET','POST'])
def imagecaption():
     if request.form['submit_button'] == 'START':
                r = requests.post(
                    "https://api.deepai.org/api/densecap",
                    files={
                        'image': open(r'C:\Users\nishc_omjn2ty\Downloads\Nmit\static\images\train.jpg', 'rb'),
                    },
                    headers={'api-key': '5a639662-a646-4656-8536-355046329996'}
                )
                print(r.json()['output']['captions'][0]['caption'])
                text = r.json()['output']['captions'][0]['caption']
                tts = gTTS((r.json()['output']['captions'][0]['caption']))
                tts.save('hello.mp3')
                speaker = win32com.client.Dispatch("SAPI.SpVoice")
                speaker.Speak(text)
                return render_template('insta_tabs.html',message=text, l=0,message_post='', display=0)
            

@app.route('/postcaption',methods=['GET','POST'])
def postcaption():
     if request.form['submit_button'] == 'START OCR':
            pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
            image_path=  r"C:\Users\nishc_omjn2ty\Downloads\Nmit\static\images\post.jpg"
            # load the original image
            image = cv2.imread(image_path)   
            ret,thresh1 = cv2.threshold(image,120,255,cv2.THRESH_BINARY)
            # pytesseract image to string to get results
            text = str(pytesseract.image_to_string(thresh1, config='--psm 6'))
            print(text)
            speaker = win32com.client.Dispatch("SAPI.SpVoice")
            speaker.Speak(text)
            return render_template('insta_tabs.html',message_post=text,message='', l=0, display=0)
            
        

            
@app.route('/videotrans',methods=['GET','POST'])
def videotrans():
    var = []
    if request.form['submit_button'] == 'START TRANSCRIPT':
#                 def print_time(result): 
#                     print(result)
#                     for i in range(len(result)):
#                         if(result[i] in arr):
#                                 ImageAddress = r"C:\Users\nishc_omjn2ty\OneDrive\Desktop\nmit2021\Video_Transcription\letters\\" + result[i]+'.jpg'
#                                 ImageItself = Image.open(ImageAddress)
#                                 ImageNumpyFormat = np.asarray(ImageItself)
#                                 ImageNumpyFormat = cv2.resize(ImageNumpyFormat,(224,224))
#                                 cv2.imshow('signs', ImageNumpyFormat)
#                                 cv2.waitKey(200)
#                         else:
#                                 cv2.waitKey(200)
#                                 continue
#                     cv2.destroyAllWindows() 

#                 arr=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r', 's','t','u','v','w','x','y','z']
#                 r = sr.Recognizer()
#                 audio = sr.AudioFile("converted.wav")

#                 # Check if camera opened successfully 
#                 if (cap.isOpened()== False):  
#                   print("Error opening video  file")

#                 i= time.perf_counter()
#                 x = 0

#                 # Read until video is completed 
#                 while(cap.isOpened()):
#                   # Capture frame-by-frame 
#                   ret, frame = cap.read() 
#                   if ret == True: 

#                     # Display the resulting frame
#                     frame = cv2.resize(frame,(512,512))
#                     cv2.imshow('Frame', frame)
#                     j = time.perf_counter()
#                     if(j-i > x):
#                         with audio as source:
#                             audio_file = r.record(source, duration=30, offset = x)
#                         try:
#                           result = r.recognize_google(audio_file)

#                         except:
#                           break

#                         _thread.start_new_thread(print_time, (result,) )
#                         print(result)
#                         var.append(result)
#                         x+=30

#                     # Press Q on keyboard to  exit 
#                     if cv2.waitKey(25) & 0xFF == ord('q'): 
#                       break

#                   # Break the loop 
#                   else:  
#                     break

#                 # When everything done, release  
#                 # the video capture object 
#                 cap.release() 

                # Closes all the frames 
                cv2.destroyAllWindows() 
                var=['recoupment 2004 with unknown fever sign I don\'t know who this guy is what was effective he had',
                     'ab started As a young rising figure in the horrible Sky was this assistance is uses of people ',
                     'how to use Kansas serve white people with side dishes like us']

                result = 'I am defined!'
            
                return render_template('insta_tabs.html',transcript=result, l = len(var), var=var, display=1)

MoviePy - Writing audio in converted.wav


MoviePy - Done.


In [ ]:
if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Mar/2021 07:56:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 07:56:30] "GET /css/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 07:56:30] "GET /static/log.jpg HTTP/1.1" 404 -


face.png written!
face.png
[]


127.0.0.1 - - [21/Mar/2021 07:56:40] "POST /upload HTTP/1.1" 302 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/images/world.png HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/globals.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/reset.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/variables.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/login.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/images/avatar.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/footer.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/explore.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/navigation.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 07:56:40] "GET /static/feed.css HTTP/1.1" 200 -
127.

the train is red


...

127.0.0.1 - - [21/Mar/2021 08:01:28] "POST /postcaption HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/images/world.png HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/reset.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/login.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/globals.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/variables.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/navigation.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/footer.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/explore.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/feed.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/profile.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:01:28] "GET /static/edit-profile.css HT

127.0.0.1 - - [21/Mar/2021 08:05:51] "GET /static/images/post.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:05:51] "GET /static/images/avatar.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:05:51] "GET /static/images/sample.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:05:51] "GET /static/images/2021-03-21-00-58-42.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:05:51] "GET /static/images/2021-03-21-00-58-42.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:05:51] "GET /static/images/sample.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:05:51] "GET /static/images/sample.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:07:57] "POST /videotrans HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:07:57] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:07:57] "GET /static/images/world.png HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:07:57] "GET /static/images/avatar.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:07:57] "GET /static/reset.css HTTP/1.1" 404

127.0.0.1 - - [21/Mar/2021 08:11:06] "GET /static/reset.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:11:06] "GET /static/images/avatar.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:11:06] "GET /static/variables.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:11:06] "GET /static/globals.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:11:06] "GET /static/login.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:11:07] "GET /static/footer.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:11:07] "GET /static/navigation.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:11:07] "GET /static/explore.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:11:07] "GET /static/feed.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:11:07] "GET /static/profile.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:11:07] "GET /static/edit-profile.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:11:07] "GET /static/mobile.css HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 08:11:07] "GET /static/images/

127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/world.png HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/avatar.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/train.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/post.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/world.png HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/avatar.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/sample.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/2021-03-21-00-58-42.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/sample.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/2021-03-21-00-58-42.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET /static/images/2021-03-21-00-58-42.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:18:46] "GET 

127.0.0.1 - - [21/Mar/2021 08:21:43] "GET /static/images/post.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:21:43] "GET /static/images/avatar.jpg HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:21:43] "GET /static/images/sample.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:21:43] "GET /static/images/2021-03-21-00-58-42.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2021 08:21:43] "GET /static/images/sample.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:21:44] "GET /static/images/2021-03-21-00-58-42.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:21:44] "GET /static/images/sample.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [21/Mar/2021 08:21:44] "GET /static/images/2021-03-21-00-58-42.mp4 HTTP/1.1" 206 -


In [3]:
import win32com.client
speaker = win32com.client.Dispatch("SAPI.SpVoice")
speaker.Speak("Jumpman Jumpman Jumpman Them boys up to something!")

1